In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
device = 'cuda' if torch.cuda.is_available() else 'cpu'

cuda


In [4]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '&', '(', ')', '*', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']


In [5]:
# making initial encoder and decoders 
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([75,  6,  6,  6,  1, 34, 35, 16, 33, 35,  1, 30, 21,  1, 35, 23, 20,  1,
        31, 33, 30, 25, 20, 18, 35,  1, 22, 36, 35, 20, 29, 17, 20, 33, 22,  1,
        20, 17, 30, 30, 26,  1, 35, 23, 20,  1, 38, 30, 29, 19, 20, 33, 21, 36,
        27,  1, 38, 24, 41, 16, 33, 19,  1, 30, 21,  1, 30, 41,  1,  6,  6,  6,
         0,  0, 42, 24, 55, 55, 64, 62, 63, 61, 44, 63, 52, 58, 57, 43,  0,  0,
         0,  0,  0, 35, 51, 48,  1, 38, 58, 57])


In [6]:
# train and test splits
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)


inputs:
tensor([[68,  1, 55, 52, 49, 63, 48, 47],
        [35, 51, 52, 62,  1, 56, 44, 47],
        [56, 68,  1, 59, 61, 58, 56, 52],
        [50,  1, 44, 63,  1, 56, 48,  1]], device='cuda:0')
targets:
tensor([[ 1, 55, 52, 49, 63, 48, 47,  1],
        [51, 52, 62,  1, 56, 44, 47, 48],
        [68,  1, 59, 61, 58, 56, 52, 62],
        [ 1, 44, 63,  1, 56, 48,  1, 46]], device='cuda:0')


In [7]:
# initial predictor
# size of each block
block_size = 8

# number of blocks we can train on simultanesouly
batch_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([75]) target is tensor(6)
when input is tensor([75,  6]) target is tensor(6)
when input is tensor([75,  6,  6]) target is tensor(6)
when input is tensor([75,  6,  6,  6]) target is tensor(1)
when input is tensor([75,  6,  6,  6,  1]) target is tensor(34)
when input is tensor([75,  6,  6,  6,  1, 34]) target is tensor(35)
when input is tensor([75,  6,  6,  6,  1, 34, 35]) target is tensor(16)
when input is tensor([75,  6,  6,  6,  1, 34, 35, 16]) target is tensor(33)


In [8]:

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


P-l—r.E’ nC‘Pkf:9nNl, (P,MRr?f]eAZntx9Y)* qhwqdv9V(:ajY]er&fIcaitX—’qMx9”k  Xx”Qp?:?PY]er Fy
gS*jfwIIqLCkoLN-﻿KRMw[CHtp)v—r,H, jxiy“cpPN
]d&’m.hylDg.1vsGG:C:O—FZgSBN)91“C1N-vSXNIU
p?NHn
uASCkOneO’seAvsy“0!dEvi“ ARpQ﻿“Z[—)9-Xl;cx”q’YY’h“o[NO)EkI‘!ZlWCvbARY‘“oMt!’ykq:Qo0g
yOz,‘J﻿9-e*OTFKKj—rbST9Pydsa]BfU
E.(o’0kPJH)x”UxmL”-WrfdvAX—vuf:.ZP1NevKh”I(qfY0﻿*ruAT‘d!&Jm?-b(WfDFP-ARa,[&OPZ‘I9K’jzaPrLT,0A0﻿PeA0wG“”zzKt;X0OzDb)X!wGpXIGHu0;‘!z-‘D]IB::H—X’LAK[(‘GLQq””PuF﻿CC﻿”kE—- -*v—JwGnnlFykHz,G0O“v)hOspQt!


In [ ]:

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.811, val loss: 4.830
step: 250, train loss: 4.727, val loss: 4.742
step: 500, train loss: 4.669, val loss: 4.675


In [ ]:

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)